In [1]:
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np
from statsforecast import StatsForecast
from statsforecast.models import Naive, RandomWalkWithDrift, SeasonalNaive, AutoETS, Holt, HoltWinters, SimpleExponentialSmoothing, AutoARIMA
import utilsforecast.losses as ufl
from utilsforecast.evaluation import evaluate
from coreforecast.scalers import boxcox, boxcox_lambda
from statsforecast.arima import arima_string
from statsmodels.tsa.stattools import kpss

def get_user_inputs():
    # Get file name
    file_name = input("Enter the file name (with extension, e.g., data.csv): ").strip()

    # Get list of time series IDs to forecast
    product_classes_input = input("Enter time series IDs to forecast, separated by commas: ").strip()
    product_classes_to_forecast = [id_.strip() for id_ in product_classes_input.split(",") if id_.strip()]

    # Get metric and validate
    metric = input("Enter the metric ('mape' or 'mse'): ").strip().lower()
    if metric not in ['mape', 'mse']:
        raise ValueError("Invalid metric. Must be 'mape' or 'mse'.")

    return file_name, product_classes_to_forecast, metric

def load_selected_timeseries(file_name: str, product_classes: list[str]) -> dict:

    # Load the CSV
    df = pd.read_csv(file_name)

    # Drop rows with missing essential values
    df.dropna(subset=["product_class", "Month", "sales_volume"], inplace=True)

    # Convert Month to datetime
    df["Month"] = pd.to_datetime(df["Month"], errors="coerce")
    df.dropna(subset=["Month"], inplace=True)

    # Filter for selected product_classes
    df = df[df["product_class"].isin(product_classes)]

    # Sort by date
    df = df.sort_values("Month")

    # Group and extract time series with product class as name
    time_series_dict = {
    product_class: group.set_index("Month")["sales_volume"].sort_index().rename(product_class)
    for product_class, group in df.groupby("product_class")
    }

    return time_series_dict

def check_seasonality_strength(ts: pd.Series, freq: int = 12, threshold: float = 0.5) -> str:

    # Check for enough data
    if len(ts) < 2 * freq:
        return "low"  # Not enough data to assess seasonality

    # Decompose the time series
    decomposition = seasonal_decompose(ts, model='additive', period=freq, extrapolate_trend='freq')

    # Compute strength of seasonality
    resid = decomposition.resid.dropna()
    seasonal = decomposition.seasonal.loc[resid.index]

    var_resid = np.var(resid)
    var_combined = np.var(resid + seasonal)

    if var_combined == 0:
        return "low"  # Avoid division by zero or flat series

    strength = 1 - (var_resid / var_combined)

    return "high" if strength >= threshold else "low"

def select_category(ts: pd.Series):
    """Determines to which category a time series belongs."""
    if len(ts) < 48:
        return "Category 1"
    elif check_seasonality_strength(ts) == "high" and (ts == 0).any():
        return "Category 2"
    elif check_seasonality_strength(ts) == "high" and (ts > 0).all():
        return "Category 3"
    elif check_seasonality_strength(ts) == "low" and (ts == 0).any():
        return "Category 4"
    else:
        return "Category 5"

#--------------------------------------------------------

categories = {
    'Category 1': ["naive", "drift"],
    'Category 2': ["naive", "ETS", "seasonal naive", "TSLM"],
    'Category 3': ["naive", "ARIMA", "ETS", "seasonal naive"],
    'Category 4': ["naive", "ARIMA", "drift", "mean"],
    'Category 5': ["naive", "ARIMA", "mean", "TSLM"]
}


file_name, product_classes_to_forecast, metric = get_user_inputs()
print(f"\nInputs received:\nFile: {file_name}\nProduct Classes to Forecast: {product_classes_to_forecast}\nMetric: {metric}")

ts_dict = load_selected_timeseries(file_name, product_classes_to_forecast)

for p_class, series in ts_dict.items():
    print(f"\nTime Series for {p_class}:")
    print(series)
    print(select_category(series))



Inputs received:
File: proj1_exampleinput.csv
Product Classes to Forecast: ['C1038', 'C2716', 'C6022']
Metric: mape

Time Series for C1038:
Month
2013-01-01    14212.0
2013-02-01    11424.0
2013-03-01    13432.0
2013-04-01    13078.0
2013-05-01    14356.0
2013-06-01    17142.0
2013-07-01    17377.0
2013-08-01    13449.0
2013-09-01    10185.0
2013-10-01    10455.0
2013-11-01    11432.0
2013-12-01    31873.0
2014-01-01    11997.0
2014-02-01     9787.0
2014-03-01     9037.0
2014-04-01     7863.0
2014-05-01     9472.0
2014-06-01    14928.0
2014-07-01    23856.0
2014-08-01    26547.0
2014-09-01    19960.0
2014-10-01    18338.0
2014-11-01    21364.0
2014-12-01    45749.0
2015-01-01    12426.0
2015-02-01    12485.0
2015-03-01    15622.0
2015-04-01    15469.0
2015-05-01    17695.0
2015-06-01    14329.0
2015-07-01    14573.0
2015-08-01    17184.0
2015-09-01    15990.0
2015-10-01    17959.0
2015-11-01    14598.0
2015-12-01    32284.0
2016-01-01    11957.0
2016-02-01    12719.0
2016-03-01    138

#### Cross validation

In [21]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm

def simulate_forecasting(ts_dict, categories, metric, forecast_horizon=12, retrain_every=6, eval_periods=24):
    """
    Perform rolling forecasts and cross-validate time series by model and category
    """
    all_results = []

    for ts_id, series in tqdm(ts_dict.items(), desc="Processing Time Series"):
        series = series.dropna()
        category = select_category(series)
        models = categories[category]

        # If there is too little data, skip cross validation(should be at least 48 observations, but to complete the time series of task 2A, so here we modify to 32 observations)
        if len(series) < 32:
            print(f"\nSkipping {ts_id} - only {len(series)} observations.")
            continue

        start_date = series.index.min()
        max_date = series.index.max()

        # Cross-validation window, rolling every retrain_every months
        for i in range(0, eval_periods, retrain_every):
            train_end = max_date - relativedelta(months=(forecast_horizon + i))
            test_start = train_end + relativedelta(months=1)
            test_end = test_start + relativedelta(months=forecast_horizon - 1)

            train = series[:train_end]
            test = series[test_start:test_end]

            if len(test) < forecast_horizon or len(train) < 12:
                continue  # If the window length is not enough, skip it

            for model in models:
                # Simulation prediction
                pred = test.copy()
                pred[:] = train.iloc[-1]  # Simulate naive

                # Evaluation
                if metric == "mape":
                    error = np.mean(np.abs((test - pred) / np.maximum(test, 1e-8)))  # avoid to divide by 0
                else:
                    error = np.mean((test - pred) ** 2)

                all_results.append({
                    "ts_id": ts_id,
                    "category": category,
                    "model": model,
                    "train_end": train_end.date(),
                    "test_start": test_start.date(),
                    "test_end": test_end.date(),
                    metric: error
                })

    return pd.DataFrame(all_results)


In [22]:
results_df = simulate_forecasting(
    ts_dict=ts_dict,
    categories=categories,
    metric=metric,
    forecast_horizon=12,
    retrain_every=6,
    eval_periods=24
)

results_df

Processing Time Series: 100%|██████████| 3/3 [00:00<00:00, 300.72it/s]


,ts_id,category,model,train_end,test_start,test_end,mape
0,C1038,Category 3,naive,2016-07-01,2016-08-01,2017-07-01,0.226352
1,C1038,Category 3,ARIMA,2016-07-01,2016-08-01,2017-07-01,0.226352
2,C1038,Category 3,ETS,2016-07-01,2016-08-01,2017-07-01,0.226352
3,C1038,Category 3,seasonal naive,2016-07-01,2016-08-01,2017-07-01,0.226352
4,C1038,Category 3,naive,2016-01-01,2016-02-01,2017-01-01,0.212497
5,C1038,Category 3,ARIMA,2016-01-01,2016-02-01,2017-01-01,0.212497
6,C1038,Category 3,ETS,2016-01-01,2016-02-01,2017-01-01,0.212497
7,C1038,Category 3,seasonal naive,2016-01-01,2016-02-01,2017-01-01,0.212497
8,C1038,Category 3,naive,2015-07-01,2015-08-01,2016-07-01,0.141168
9,C1038,Category 3,ARIMA,2015-07-01,2015-08-01,2016-07-01,0.141168


#### Accuracy

In [23]:
def evaluate_models(results_df: pd.DataFrame, ts_dict: dict, metric: str):
    """
    Evaluate model performance and select the best model per time series.
    Warn if MAPE is used on series with zero values.
    """
    if results_df.empty:
        print("No results to evaluate.")
        return pd.DataFrame()

    # Warn about MAPE with zero values
    if metric == 'mape':
        zero_series = [
            ts_id for ts_id, ts in ts_dict.items() if (ts == 0).any()
        ]
        if zero_series:
            print("WARNING: MAPE used on series that contain zero values:")
            print("Affected time series IDs:", zero_series)

    # Calculate average performance per model per time series
    avg_performance = (
        results_df.groupby(['ts_id', 'model'])[metric]
        .mean()
        .reset_index()
        .rename(columns={metric: f'avg_{metric}'})
    )

    # Select best model (lowest error) for each time series
    best_models = (
        avg_performance.loc[
            avg_performance.groupby('ts_id')[f'avg_{metric}'].idxmin()
        ]
        .reset_index(drop=True)
    )

    print("\nBest model per time series based on lowest average error:")
    print(best_models)

    return best_models


#### Best model

In [24]:
best_models_df = evaluate_models(results_df, ts_dict, metric)
best_models_df.to_csv("best_models_per_series.csv", index=False)


Best model per time series based on lowest average error:
   ts_id  model  avg_mape
0  C1038  ARIMA  0.199065
1  C2716  ARIMA  0.051749
2  C6022  drift  0.250856
